In [ ]:
import pandas as pd
import os

# =============================================================================
# 1. DOSYA İSİMLERİ (Colab'e yüklediğin dosyalarla aynı olmalı)
# =============================================================================
FILE_1 = "labeled_songs_progress.csv"       # İlk büyük veri seti
FILE_2 = "labeled_songs_progress2.csv"      # İkinci büyük veri seti
FILE_3 = "sadece_mutluluk_sarkilari2.csv"   # Sadece mutluluk içeren küçük set

OUTPUT_FILE = "final_egitim_seti.csv"       # Çıktı dosyasının adı
MAX_SAMPLES = 6000                          # Sınıf başına maksimum şarkı limiti

# =============================================================================
# 2. VERİLERİ YÜKLEME VE STANDARTLAŞTIRMA
# =============================================================================
print("Dosyalar okunuyor...")

# Dosya 1 ve 2 zaten uyumlu formatta
df1 = pd.read_csv(FILE_1)
df2 = pd.read_csv(FILE_2)

# Dosya 3 (Mutluluk) sütun isimleri farklı, onları düzeltiyoruz
df3 = pd.read_csv(FILE_3)
df3 = df3.rename(columns={'song': 'title', 'singer': 'artist'})

# Sadece eğitim için gerekli sütunları seçiyoruz
cols = ['title', 'artist', 'lyrics', 'label', 'score']
df1 = df1[cols]
df2 = df2[cols]
df3 = df3[cols]

# =============================================================================
# 3. BİRLEŞTİRME VE TEMİZLİK
# =============================================================================
print("Veriler birleştiriliyor...")
# Hepsini alt alta ekle
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Şarkı sözleri (lyrics) birebir aynı olan tekrarları sil
print(f"Tekilleştirme öncesi toplam satır: {len(combined_df)}")
combined_df = combined_df.drop_duplicates(subset=['lyrics'])
print(f"Tekilleştirme sonrası toplam satır: {len(combined_df)}")

# =============================================================================
# 4. DENGELEME (BALANCING) STRATEJİSİ
# =============================================================================
print(f"\nDengeleme Başlıyor (Hedef Tavan: {MAX_SAMPLES})...")

balanced_dataframes = []

for label in combined_df['label'].unique():
    # O sınıfa ait tüm verileri çek
    class_subset = combined_df[combined_df['label'] == label]
    count = len(class_subset)

    if count > MAX_SAMPLES:
        # LİMİTİ AŞIYORSA: En yüksek 'score' (güven) puanına sahip olanları seç
        # Bu sayede verinin "kaymağını" almış oluruz.
        selected = class_subset.sort_values(by='score', ascending=False).head(MAX_SAMPLES)
        print(f"  -> '{label}': {count} taneden {MAX_SAMPLES} taneye indirildi. (En iyiler seçildi)")
    else:
        # LİMİTİN ALTINDAYSA: Hepsini koru (Mutluluk gibi sınıflar için)
        selected = class_subset
        print(f"  -> '{label}': {count} tane korundu.")

    balanced_dataframes.append(selected)

# =============================================================================
# 5. SONUÇ VE KAYIT
# =============================================================================
# Parçaları birleştir ve karıştır (Shuffle)
final_df = pd.concat(balanced_dataframes).sample(frac=1, random_state=42).reset_index(drop=True)

print("\n------------------------------------------------")
print("FİNAL VERİ SETİ DAĞILIMI:")
print(final_df['label'].value_counts())
print("------------------------------------------------")

# Kaydet
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ İşlem Tamamlandı! Dosya kaydedildi: {OUTPUT_FILE}")
print("Dosyalar sekmesinden indirip projene başlayabilirsin.")

Dosyalar okunuyor...
Veriler birleştiriliyor...
Tekilleştirme öncesi toplam satır: 80012
Tekilleştirme sonrası toplam satır: 79302

Dengeleme Başlıyor (Hedef Tavan: 6000)...
  -> 'öfke ve isyan': 39789 taneden 6000 taneye indirildi. (En iyiler seçildi)
  -> 'efkar ve melankoli': 7224 taneden 6000 taneye indirildi. (En iyiler seçildi)
  -> 'umut ve motivasyon': 13989 taneden 6000 taneye indirildi. (En iyiler seçildi)
  -> 'aşk ve romantizm': 15869 taneden 6000 taneye indirildi. (En iyiler seçildi)
  -> 'mutluluk ve eğlence': 2431 tane korundu.

------------------------------------------------
FİNAL VERİ SETİ DAĞILIMI:
label
efkar ve melankoli     6000
öfke ve isyan          6000
aşk ve romantizm       6000
umut ve motivasyon     6000
mutluluk ve eğlence    2431
Name: count, dtype: int64
------------------------------------------------

✅ İşlem Tamamlandı! Dosya kaydedildi: final_egitim_seti.csv
Dosyalar sekmesinden indirip projene başlayabilirsin.


In [ ]:
# =============================================================================
# BÖLÜM 1: KURULUM VE VERİ ÖN İŞLEME (DATA PIPELINE)
# =============================================================================

# 1. Gerekli Kütüphaneleri Kur (Eğer kurulu değilse)
!pip install transformers torch scikit-learn pandas -q

import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Cihazı Seç (GPU varsa kullan, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Kullanılan Cihaz: {device}")

# 2. Ayarlar (Hyperparameters)
MODEL_NAME = "dbmdz/bert-base-turkish-cased" # Türkçenin en iyi modellerinden biri
MAX_LEN = 256   # Analizimize göre 256 kelime çoğu şarkı için yeterli ve hızlı
BATCH_SIZE = 16 # GPU hafızasına göre 16 veya 32 idealdir.
EPOCHS = 4      # Fine-tuning için genelde 2-4 epoch yeterlidir.

# 3. Veriyi Yükle
df = pd.read_csv("final_egitim_seti.csv")

# 4. Etiketleri Sayısala Çevir (Label Encoding)
# Model metin (string) anlamaz, sayı (0,1,2..) ister.
label_encoder = LabelEncoder()
df['label_id'] = label_encoder.fit_transform(df['label'])

# Etiket haritasını görelim (Hangi sayı hangi duygu?)
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"\n🏷️ Etiket Haritası: {label_map}")

# 5. Veriyi Eğitim ve Test Olarak Böl (Stratified Split)
# Stratify=True diyerek her duygudan eğitim ve test setine eşit oranda dağıtıyoruz.
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label_id.values,
    test_size=0.15, # %15 Test (Doğrulama), %85 Eğitim
    random_state=42,
    stratify=df.label_id.values
)

df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

print(f"\n📊 Veri Bölümleme: Eğitim={len(X_train)}, Doğrulama={len(X_val)}")

# 6. Sınıf Ağırlıklarını Hesapla (Class Weights) - KRİTİK ADIM
# Mutluluk az olduğu için ona daha yüksek ağırlık vereceğiz.
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"⚖️ Sınıf Ağırlıkları: {class_weights}")
# Örnek: Eğer bir sınıfın ağırlığı 2.5 ise, model o sınıfı yanlış bilince 2.5 kat daha fazla ceza alır.

# 7. Özel Dataset Sınıfı (Profesyonel Yapı)
class SentimentDataset(Dataset):
    def __init__(self, tokenizer, data, max_len):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = max_len
        self.lyrics = data.lyrics.values
        self.targets = data.label_id.values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        lyric = str(self.lyrics[index])

        # BERT Tokenizer'ı kullanıyoruz
        inputs = self.tokenizer.encode_plus(
            lyric,
            None,
            add_special_tokens=True, # [CLS] ve [SEP] ekle
            max_length=self.max_len,
            padding='max_length',    # Sabit uzunluğa tamamla
            return_token_type_ids=True,
            truncation=True,         # Uzunsa kes
            return_attention_mask=True,
            return_tensors='pt'      # PyTorch Tensoru döndür
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

# 8. Tokenizer ve DataLoader Hazırlığı
print("\n⚙️ Tokenizer Yükleniyor ve Veri Loader Hazırlanıyor...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

training_set = SentimentDataset(tokenizer, df[df.data_type=='train'], MAX_LEN)
validation_set = SentimentDataset(tokenizer, df[df.data_type=='val'], MAX_LEN)

train_params = {'batch_size': BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
val_params = {'batch_size': BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)

# --- SANITY CHECK (SAĞLAMASI) ---
# İlk batch'i çekip boyutlarına bakalım. Her şey yolunda mı?
sample_batch = next(iter(training_loader))
print("\n✅ KONTROL (SANITY CHECK):")
print(f"Batch Input IDs Boyutu: {sample_batch['input_ids'].shape}")
print(f"Batch Attention Mask Boyutu: {sample_batch['attention_mask'].shape}")
print(f"Batch Targets Boyutu: {sample_batch['targets'].shape}")
print("Veri hazırlığı başarıyla tamamlandı!")

✅ Kullanılan Cihaz: cuda

🏷️ Etiket Haritası: {'aşk ve romantizm': np.int64(0), 'efkar ve melankoli': np.int64(1), 'mutluluk ve eğlence': np.int64(2), 'umut ve motivasyon': np.int64(3), 'öfke ve isyan': np.int64(4)}

📊 Veri Bölümleme: Eğitim=22466, Doğrulama=3965
⚖️ Sınıf Ağırlıkları: tensor([0.8810, 0.8810, 2.1748, 0.8810, 0.8810], device='cuda:0')

⚙️ Tokenizer Yükleniyor ve Veri Loader Hazırlanıyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]


✅ KONTROL (SANITY CHECK):
Batch Input IDs Boyutu: torch.Size([16, 256])
Batch Attention Mask Boyutu: torch.Size([16, 256])
Batch Targets Boyutu: torch.Size([16])
Veri hazırlığı başarıyla tamamlandı!


In [ ]:
# =============================================================================
# BÖLÜM 2: MODEL MİMARİSİ VE EĞİTİM MOTORU (DÜZELTİLMİŞ VERSİYON)
# =============================================================================
from transformers import BertModel, get_linear_schedule_with_warmup
import torch.nn as nn
from torch.optim import AdamW # DEĞİŞİKLİK: AdamW artık torch'tan çağrılıyor

# 1. Profesyonel Model Sınıfı (Custom BERT Class)
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]
        output = self.drop(pooled_output)
        return self.out(output)

# 2. Modeli Başlatma ve GPU'ya Aktarma
print("🧠 Model inşa ediliyor ve GPU'ya yükleniyor...")
model = SentimentClassifier(len(label_map))
model = model.to(device)

# 3. Optimizer ve Scheduler Ayarları
# DEĞİŞİKLİK: correct_bias parametresi kaldırıldı (PyTorch AdamW'da yoktur)
optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(training_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# 4. Kayıp Fonksiyonu (Dengesiz Veri Korumalı)
loss_fn = nn.CrossEntropyLoss(weight=class_weights).to(device)

# 5. Yardımcı Fonksiyon: Bir Epoch EĞİTİM (Train)
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

# 6. Yardımcı Fonksiyon: DEĞERLENDİRME (Validation)
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

print("✅ Model ve Eğitim Fonksiyonları Hazır!")

🧠 Model inşa ediliyor ve GPU'ya yükleniyor...


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

✅ Model ve Eğitim Fonksiyonları Hazır!


In [ ]:
# =============================================================================
# BÖLÜM 3: EĞİTİM DÖNGÜSÜ (THE TRAINING LOOP)
# =============================================================================
from collections import defaultdict
import numpy as np

# Eğitimi başlatıyoruz
history = defaultdict(list)
best_accuracy = 0

print(f"🚀 Eğitim Başlıyor! Toplam {EPOCHS} Epoch sürecek...\n")

for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    # 1. Eğitim Aşaması (Train)
    train_acc, train_loss = train_epoch(
        model,
        training_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df[df.data_type=='train'])
    )

    print(f'Eğitim Hatası (Loss): {train_loss:.4f} | Eğitim Başarısı (Acc): {train_acc*100:.2f}%')

    # 2. Değerlendirme Aşaması (Validation)
    val_acc, val_loss = eval_model(
        model,
        validation_loader,
        loss_fn,
        device,
        len(df[df.data_type=='val'])
    )

    print(f'Test   Hatası (Loss): {val_loss:.4f} | Test   Başarısı (Acc): {val_acc*100:.2f}%')

    # Geçmişi kaydet (Grafik çizdirmek için)
    history['train_acc'].append(train_acc.item()) # Tensor'u sayıya çevir
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.item())
    history['val_loss'].append(val_loss)

    # 3. En İyi Modeli Kaydetme (Checkpoint)
    # Eğer bu epoch'taki test başarısı, şimdiye kadarki en iyisiyse kaydet.
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc
        print("⭐ Tebrikler! Model gelişti ve kaydedildi.")

    print() # Boşluk bırak

print(f"\n✅ EĞİTİM TAMAMLANDI! En yüksek test başarısı: {best_accuracy*100:.2f}%")

🚀 Eğitim Başlıyor! Toplam 4 Epoch sürecek...

Epoch 1/4
----------
Eğitim Hatası (Loss): 0.7953 | Eğitim Başarısı (Acc): 70.93%
Test   Hatası (Loss): 0.5442 | Test   Başarısı (Acc): 81.66%
⭐ Tebrikler! Model gelişti ve kaydedildi.

Epoch 2/4
----------
Eğitim Hatası (Loss): 0.4100 | Eğitim Başarısı (Acc): 86.53%
Test   Hatası (Loss): 0.5488 | Test   Başarısı (Acc): 84.41%
⭐ Tebrikler! Model gelişti ve kaydedildi.

Epoch 3/4
----------
Eğitim Hatası (Loss): 0.2184 | Eğitim Başarısı (Acc): 93.74%
Test   Hatası (Loss): 0.6480 | Test   Başarısı (Acc): 84.62%
⭐ Tebrikler! Model gelişti ve kaydedildi.

Epoch 4/4
----------
Eğitim Hatası (Loss): 0.1243 | Eğitim Başarısı (Acc): 96.95%
Test   Hatası (Loss): 0.7704 | Test   Başarısı (Acc): 85.32%
⭐ Tebrikler! Model gelişti ve kaydedildi.


✅ EĞİTİM TAMAMLANDI! En yüksek test başarısı: 85.32%
